In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os

import textwrap

engine = create_engine("postgresql://localhost/mimic?user=xxxx&password=xxxx") 
# user name and password needs to be modified based on the local setup of the postgress server  

path_ft = "/home/ft/fastText-0.2.0/fasttext"## modify path to fastText 
path_model_bin = "T300SG.bin"## modify path to bin

In [ ]:
total_row = 58929 ## use a few rows to test 
rowlimit = total_row
fname = "discharge_summary.txt"
gname = "h.csv" ## hospital admission 


with open(gname, "a") as g:
    g.write('HADM_ID\n')


df = pd.read_sql_query(
    'select HADM_ID, text from mimic.mimiciii.noteevents where category=\'Discharge summary\'  offset ' + 
    str(rowlimit) + ' limit ''' + str(rowlimit),
    con=engine)  # change limit number
df = df.sort_values('HADM_ID').reset_index()
df1 = df.groupby('HADM_ID').agg(lambda x: ''.join(x.unique()))
df1 = df1.reset_index()

with open(fname, "a") as f, open(gname, "a") as g:
    for index, line in df1.iterrows():
        t = " ".join(line[1].split())
        f.write(t  + '\n')
        g.write(str(line[0]) + '\n')


In [ ]:
## fastText processing
vout_fname = "fasttextout.txt"
cmd = "{} print-sentence-vectors {} < {} > {}".format(path_ft, path_model_bin, fname, vout_fname)
os.system(cmd)


In [ ]:
## processing the fastText output. The code here includes options to print outputs and save output. 
## Please comment out if not needed.

TEMP = "temp.txt"

vectors_file = open(TEMP, "w+")
with open(vout_fname) as f:
    line = f.readline()
    while(line):
        line = line.replace(" ", ",")
        vectors_file.write(line[:-2])
        vectors_file.write("\n")
        line = f.readline()
vectors_file.close()
df_vectors = pd.read_csv(TEMP, header=None, index_col=None)
df_vectors.rename(columns=lambda x: "vec" + str(x+1), inplace=True)
df_vectors.rename(columns={'':''}, inplace=True)
df_vectors.to_csv('vector_f.csv', index=False)
print(df_vectors)

In [ ]:
with open('h.csv') as f1, open('vector_f.csv') as f2, open('vector_T300SG.csv','w') as out:
    for x in f1:
        y = next(f2)
        out.write("{},{}".format(x.strip(),y))